In [14]:
from tensorflow.python.client import device_lib

local_device_protos = device_lib.list_local_devices()

print(local_device_protos)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5613279067707206342
xla_global_id: -1
]


In [15]:
from keras.layers import Input, Embedding, Dense, Reshape, Concatenate, Conv2D, LeakyReLU, Flatten, Dropout, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam

In [16]:
def define_discriminator(in_shape=(224, 224, 1), n_classes=144):  # providing class info
    # label input
    in_label = Input(shape=(1,))  # label input
    # embedding for categorical input (each label represented by a vector of size 50)
    # takes the class label and maps it to a random vector of size 50
    li = Embedding(n_classes, 50)(in_label)
    n_nodes = in_shape[0] * in_shape[1]  # 224x224=50176
    # labels have to be the same size as the image
    # 50176x1
    li = Dense(n_nodes)(li)
    li = Reshape((in_shape[0], in_shape[1], 1))(li)

    # Image input
    in_image = Input(shape=in_shape)  # image input 32x32x3

    # Concat label and image
    # 4 channels, 3 for image and the other for labels
    merge = Concatenate()([in_image, li])  # 32x32x4

    #  this part is the same an uncoditional GAN up to the output layer
    fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Flatten()(fe)
    fe = Dropout(0.4)(fe)

    out_layer = Dense(1, activation='sigmoid')(fe)
    # combine input label with input img
    model = Model([in_image, in_label], out_layer, name='cgan_discriminator')
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

    return model

In [17]:
# test = define_discriminator()

# test.summary()

# from keras.utils.vis_utils import plot_model
# plot_model(test, to_file='cgan_discriminator2.png', show_shapes=True, show_layer_names=True)

In [18]:
def define_generator(latent_dim, n_classes=144):
    # Label input
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)  # shape (1, 50)
    n_nodes = 28 * 28  # 784 - to match the dimensions for concat later
    li = Dense(n_nodes)(li)
    li = Reshape((28, 28, 1))(li)  # this is out label input for the model

    # Latent vector input
    in_lat = Input(shape=(latent_dim,))
    n_nodes = 128 * 28 * 28  # 100352
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((28, 28, 128))(gen)

    merge = Concatenate()([gen, li])  # merge image gen and label inputs
    gen = Conv2DTranspose(128, (4, 4), strides=(
        2, 2), padding='same')(merge)  # upsample to 56x56
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Conv2DTranspose(128, (4, 4), strides=(
        2, 2), padding='same')(gen)  # upsample to 112x112
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Conv2DTranspose(128, (4, 4), strides=(
        2, 2), padding='same')(gen)  # upsample to 224x224
    gen = LeakyReLU(alpha=0.2)(gen)

    out_layer = Conv2D(1, (7, 7), activation='tanh', padding='same')(
        gen)  # output layer 224x224x1
    model = Model([in_lat, in_label], out_layer, name='cgan_generator')

    return model

In [19]:
test = define_generator(100)
test.summary()

NameError: name 'test' is not defined

In [ ]:
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

In [ ]:
def load_real_samples(batch_size=64):
    datagen = ImageDataGenerator()
    sdir_gray = r'./data_gray/'
    train_dir = os.path.join(sdir_gray, 'train_gray')
    train_it = datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),  # adjust this to the size of your images
        batch_size=batch_size,  # number of images to load at a time
        class_mode='categorical',  # use 'categorical' for multi-class problems
        color_mode='grayscale'  # or 'rgb' TODO: make an if
    )

    # store all images and labels in a list
    X = []
    y = []
    for i in range(len(train_it)):
        print(1)
        images, labels = train_it.next()
        X.append(images)
        # extract label numbers
        label_numbers = np.argmax(labels, axis=1)
        y.append(label_numbers)

    y = [np.array(batch_labels) for batch_labels in y]
    y = np.concatenate(y)
    y = y.reshape(-1, 1)  # reshape y to have shape (11520, 1)

    X = np.concatenate(X)
    X = X.astype('float32')
    X = (X - 127.5) / 127.5  # normalize to [-1, 1]

    return [X, y]


def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = np.random.randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    y = np.ones((n_samples, 1))  # generate 'real' class labels (1)

    return [X, labels], y  # [image, label], real


# latent points = latent vector + label
def generate_latent_points(latent_dim, n_samples, n_classes=144):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)  # TODO: print this shape
    # generate labels
    labels = np.random.randint(0, n_classes, n_samples)

    return [z_input, labels]  # random vector + random label


def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))

    return [images, labels_input], y  # [image, label], fake


def print_dataset_in_file(dataset):
    print(dataset[0].shape)  # (11520, 224, 224, 1)
    print(len(dataset[0]))
    print(dataset[1].shape)  # (11520, 1)
    print(len(dataset[1]))

    with open('output.txt', 'w') as f:
        for i in range(len(dataset[1])):
            print(dataset[0][i], file=f)
            print(dataset[1][i], file=f)
        f.flush()

In [ ]:
def define_gan(g_model, d_model):
    d_model.trainable = False  # discriminator is trained separately, so make it non-trainable
    
    ## connect generator and discriminator
    # first get noise and label inputs from generator
    gen_noise, gen_label = g_model.input # latent vector size and label size
    # get image output from generator
    gen_output = g_model.output
    
    # generate iamge output corresponding to the label
    gan_output = d_model([gen_output, gen_label])
    # define GAN model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=64):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    for i in range(n_epochs):  # enumerate epochs
        for j in range(bat_per_epo):  # enumerate batches
            [X_real, labels_real], y_real = generate_real_samples(
                dataset, half_batch)

            d_loss_real, _ = d_model.train_on_batch(
                [X_real, labels_real], y_real)  # train D on real images

            [X_fake, labels], y_fake = generate_fake_samples(
                g_model, latent_dim, half_batch)

            d_loss_fake, _ = d_model.train_on_batch(
                [X_fake, labels], y_fake)  # train D on fake images

            [z_input, labels_input] = generate_latent_points(
                latent_dim, n_batch)  # generator input
            # label generated images as true to fake discriminator
            y_gan = np.ones((n_batch, 1))

            # train G on fake images with inverted labels
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)

            # print losses on this batch
            print('epoch>%d, batch:%d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                  (i+1, j+1, bat_per_epo, d_loss_real, d_loss_fake, g_loss))

    # save the generator model
    g_model.save('cgan_generator_clock.h5')

In [ ]:
import numpy as np
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
# train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=500)

c:\Users\dimau\Anaconda3\envs\clock\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 11520 images belonging to 144 classes.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


TypeError: float() argument must be a string or a number, not 'JpegImageFile'